[Oregon Curriculum Network](http://4dsolutions.net/ocn/)

# The School of Tomorrow

[Home Page](School_of_Tomorrow.ipynb)

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/49982660476/in/dateposted-public/" title="ramanujan1"><img src="https://live.staticflickr.com/65535/49982660476_aa975f3e3e_c.jpg" width="800" height="330" alt="ramanujan1"></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

(scroll down for embedded source code and more remarks about its runtime context)

# Number Theory

To what extent was Number Theory purged from some curricula owing to anti-German sentiment and a wish to deprecate all things German?  Too much Gauss would never do, and yet he was a Principal.  Solution:  spend more time on Euclidean Geometry instead.  But did it have to be either / or?

Sometimes our Notebook will be more of a springboard, a diving board, into a deep well or chasm (sounds dramatic).  You may jump into Number Theory through Cryptography as well, and may wish to study them together.  Indeed, what has brought Number Theory back into vogue, if it ever went away, is the ubiquity (omnipresence) of cryptography in our daily lives today.

You'll want to learn about Fermat's Little Theorem, about totatives and totients, and about Diophantine Equations.  Even earlier, let's start out with Euclid's Method for the GCD (greatest common divisor), as one of the earliest numeric algorithms out there.

# Euclid's Method

I had you two lengths and ask you what size length in whole number size, will divide into both with no remainder.  Start with the two lengths also being whole number.  Everything in this picture is an integer, a positive integer, a length.  What integer length divides evenly into these two?  Clearly 1 does, as 1 divides into everything.  How about 2?  How about 3?  What's the greatest number that divides them both, without remainder?

Well, what if the smaller length already divides the larger into two.  If we call them A and B and divide A into B, not knowing which is longer, then if A is longer, it won't go into B at all.  If it does, does it go just once?  If so, we're done, and A is the GCD.  Otherwise, last possibility, there's some remainder.

That remainder then, is the next shorter length of interest.  If there's a GCD > 1, it will divide both of these, as if it does, it will also divide into B, which consists of N As + R for remainder.  So make R the new A, and B the new longest length, what used to be A.

In [1]:
N, R = divmod(12, 3) # how many times goes in N, what remainder R
(N, R)

(4, 0)

In [2]:
divmod(3, 12) # 12 doesn't go into 3, 3 is left

(0, 3)

Here's a recursive solution, based on the description above:

In [3]:
def GCD(A, B):
    N, R = divmod(B, A)  # bigger by smaller?
    if N == 0:           # if not
        return GCD(B, A) # then switch
    elif R == 0:
        return A         # if no remainder, we're done
    elif R == 1:
        return 1         # down to 1, nowhere to go so 1
    elif R > 0:
        return GCD(R, B) # keep drilling down

In [4]:
GCD(100, 30)

10

In [5]:
GCD(81, 18)

9

However Guido came up with a more brilliant, non-recursive solution, that automatically switches the arguments if a > b:

In [6]:
18 % 81 # 81 doesn't go into 18, 18 remains

18

In [7]:
81 % 18 # 18 goes into 81 9 times

9

In [8]:
12 % 24 # 24 doesn't go into 12, 12 is the remainder

12

In [9]:
def gcd(a, b):
    while b:
        b, a = a % b, b
    return a

In [10]:
gcd(18, 81)

9

In [11]:
gcd(100, 30)

10

In [12]:
gcd(12, 3)

3

In [13]:
gcd(3, 12)

3

Finally, how about the Lowest Common Multiple or LCM.  That turns out to be the product of A and B (all factors combined) divided by the GCD (all factors common to both).

In [14]:
def lcm(a, b):
    return int((a * b)/gcd(a, b))

In [15]:
lcm(11, 9)

99

We can use that to define a Fraction, or rational number (Rat).  Our adventures in Python's object oriented model might, in fact, begin here, even though Python's standard library already includes a Fraction class in fraction.  Our tests might compare their behaviors.

In [16]:
from fractions import Fraction

In the Rational Number or alternative fraction class below, we make use of the ```gcd``` and ```lcm``` functions defined above.

If you haven't looked at Python a lot, you may miss some of what's going on:  Python's special names define the guts of a fraction [in terms of its operations](https://docs.python.org/3/library/operator.html). 

When you add two fractions, what happens?  A lowest common multiple makes for a denominator.  Given the initialization process reduces the fraction to lowest terms, just multiplying the two denominators together would also work.

Complexity arises when we let integers play with Rationals, which we should, as they're a subset of same.  $N \subset Z \subset Q \subset R \subset C$ as we say.

In [18]:
from __future__ import annotations

class Rat:
    
    def __init__(self, n, d=None):
        
        if type(d) is None:
            d = 1
        if d == 0:
            raise ZeroDivisionError
            
        _gcd = gcd(n, d)
        self.numer = n//_gcd
        self.denom = d//_gcd
        
    def __eq__(self, other):
        if type(other) is int:
            return ((self.numer) == other 
                     and self.denom == 1)
        return ((self.numer == other.numer) 
                and (self.denom == other.denom))
        
    def __repr__(self):
        return "Rat(%s, %s)" % (self.numer, self.denom)
    
    def __add__(self, other) -> Rat:
        # get an lcm before adding
        if type(other) is int:
            other = Rat(other, 1)
        _lcm = lcm(self.denom, other.denom)
        return Rat(self.numer * _lcm//self.denom
               + other.numer * _lcm//other.denom, 
               _lcm) # denominator
    
    def __sub__(self, other) -> Rat:
        # add the additive inverse of other
        if type(other) is int:
            other = Rat(other, 1)
        return self + (-other)
    
    def __neg__(self) -> Rat:
        # return the additive inverse of self
        return Rat(-self.numer, self.denom)
    
    def __mul__(self, other) -> Rat:
        # multiply two rationals
        if type(other) is int:
            other = Rat(other, 1)
        return Rat(self.numer * other.numer, 
                  self.denom * other.denom)
    
    def __truediv__(self, other) -> Rat:
        # multiply by the multiplicative inverse of other
        if type(other) is int:
            other = Rat(other, 1)
        return self * other**(-1)
    
    def __pow__(self, n : int) -> Rat:
        # raise self to the nth power
        return Rat(self.numer ** n, self.denom ** n)
    
    __rmul__ = __mul__
    __radd__ = __add__
    __rtruediv__ = __truediv__

In [19]:
from unittest import TestCase
import unittest

class TestRat(TestCase):
    
    def test_1(self):
        p = Rat(1, 3)
        q = Rat(2, 3)
        self.assertEqual(p + q, 1)
        
    def test_2(self):
        fp = Fraction(16,32)
        fq = Fraction(18,56)
        p = Rat(16,32)
        q = Rat(18,56)
        fproduct = fp * fq
        product = p * q
        self.assertEqual(fproduct.numerator, product.numer)
        self.assertEqual(fproduct.denominator, product.denom)
        
    def test_3(self):
        p = Rat(16,32)
        q = Rat(18,56)
        result = p - q
        fp = Fraction(16,32)
        fq = Fraction(18,56)
        fresult = fp - fq
        self.assertEqual(fresult.numerator, result.numer)
        self.assertEqual(fresult.denominator, result.denom)        

In [20]:
unittest.main(argv=[''], verbosity=3, exit=False)

test_1 (__main__.TestRat) ... ok
test_2 (__main__.TestRat) ... ok
test_3 (__main__.TestRat) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.026s

OK


## RAMANUJAN

\begin{equation*}
\frac{1}{\Bigl(\sqrt{\phi \sqrt{5}}-\phi\Bigr) e^{\frac25 \pi}} =
1+\frac{e^{-2\pi}} {1+\frac{e^{-4\pi}} {1+\frac{e^{-6\pi}}
{1+\frac{e^{-8\pi}} {1+\ldots} } } }
\end{equation*}

In [22]:
# %load genius_identity.py
#!/usr/bin/env python3
"""
Created on Mon Jun  8 17:48:56 2020

@author: Kirby Urner

Verifying a Ramanujan Identity
https://flic.kr/p/2j9NcAd

Keywords:  recursion, number theory, arbitrary precision

School of Tomorrow
https://medium.com/@kirbyurner/calculator-of-tomorrow-using-arbitrary-precision-8f219b0092d9
https://repl.it/@kurner/RamanujanIdentity01
"""

import mpmath
from mpmath import e, pi
print(mpmath.libmp.BACKEND)
mpmath.mp.dps = 100
two   = mpmath.mpf('2')
five  = mpmath.mpf('5')
root5 = mpmath.mpf('5').sqrt()
phi = (1 + root5)/2

term = "(1 + (e ** (-2*{} * pi))/{})"

def cont_frac(n, c=1):
    """
    Recursively build the continued fraction, 
    as a string, to level of depth n
    """
    if n==0:
        return "1"
    else:    
        return "(1 + ((e ** (-2*{} * pi)/{})))".format(c, cont_frac(n-1, c+1))

print(eval(cont_frac(10))) # evaluate the continued fraction
print(1/(((phi * root5).sqrt() - phi) * e ** (two/five * pi))) # left side of the equation
print("Ta daa!")

gmpy
1.001867436219318606077227680424157087122424127427497054500130190210949798909562825712938250353099963
1.001867436219318606077227680424157087122424127427497054500130190210949798909562825712938250353099963
Ta daa!
